## Lab9 UCM EN RAG TTS Demo

In [1]:
import os

print(os.environ['HSA_OVERRIDE_GFX_VERSION'])

In [2]:
import os
os.environ['HIP_VISIABLE_DEVICES'] = "0"

In [3]:
import torch

In [4]:
print(torch.__version__)

2.4.0.dev20240401+rocm6.0


In [5]:
# Query GPU
if torch.cuda.is_available():
    device = torch.device("cuda")          # a CUDA device object
    print('Using GPU:', torch.cuda.get_device_name(0))
    print('GPU properties:', torch.cuda.get_device_properties(0))
else:
    device = torch.device("cpu")
    print('Using CPU')

Using GPU: AMD Radeon PRO W7900
GPU properties: _CudaDeviceProperties(name='AMD Radeon PRO W7900', major=11, minor=0, gcnArchName='gfx1100', total_memory=46064MB, multi_processor_count=48)


In [6]:
# NOTE: This is ONLY necessary in jupyter notebook.
# Details: Jupyter runs an event-loop behind the scenes.
#          This results in nested event-loops when we start an event-loop to make async queries.
#          This is normally not allowed, we use nest_asyncio to allow it for convenience.

import nest_asyncio

nest_asyncio.apply()

In [7]:
from llama_index.core import VectorStoreIndex, SimpleDirectoryReader, Settings
from llama_index.core.node_parser import SentenceSplitter

from llama_index.embeddings.ollama import OllamaEmbedding
from llama_index.llms.ollama import Ollama

# Set embedding model
# Please download it ahead running this lab by "ollama pull nomic-embed-text"
#Settings.embed_model = OllamaEmbedding(model_name="nomic-embed-text")
Settings.embed_model = OllamaEmbedding(model_name="mxbai-embed-large")

# Set ollama model
Settings.llm = Ollama(model="llama3", request_timeout=120.0)

In [8]:
import chromadb
from llama_index.core import VectorStoreIndex
from llama_index.vector_stores.chroma import ChromaVectorStore
from llama_index.core import StorageContext

if not os.path.exists("./chroma_db/CUM_en_db"):
    # initialize client
    db = chromadb.PersistentClient(path="./chroma_db/CUM_en_db")
    # get collection
    chroma_collection = db.get_or_create_collection("CUM_en_db")
    # assign chroma as the vector_store to the context
    vector_store = ChromaVectorStore(chroma_collection=chroma_collection)
    storage_context = StorageContext.from_defaults(vector_store=vector_store)
    # Load data
    documents = SimpleDirectoryReader(input_files=["../data/FordUM.pdf"]).load_data()
    # Build vector index per-document
    index = VectorStoreIndex.from_documents(
        documents,
        storage_context=storage_context,
        transformations=[SentenceSplitter(chunk_size=1024, chunk_overlap=50)],
    )
else:
    # initialize client
    db = chromadb.PersistentClient(path="./chroma_db/CUM_en_db")
    # get collection
    chroma_collection = db.get_or_create_collection("CUM_en_db")
    # assign chroma as the vector_store to the context
    vector_store = ChromaVectorStore(chroma_collection=chroma_collection)
    storage_context = StorageContext.from_defaults(vector_store=vector_store)
    # load your index from stored vectors
    index = VectorStoreIndex.from_vector_store(
        vector_store, storage_context=storage_context
    )

In [9]:
# create a query engine
query_engine = index.as_query_engine(streaming=True, response_mode="compact", similarity_top_k=3)

In [10]:
# Query Test 0
response = query_engine.query("Which pages could get the detials about Exterior Lighting Control?")
response.print_response_stream()

Based on the provided context information, I would suggest that you can find details about Exterior Lighting Control on page 657 or page 124.

In [11]:
# Q
response = query_engine.query("What's the steps to SWITCHING THE DAYTIME RUNNING LAMPS ON AND OFF and which pages show it?")
response.print_response_stream()

The daytime running lamps can be switched on and off using the following steps:

1. Adjust the front camera's brightness by cleaning the windshield if the visibility is reduced.
2. Wait a short period for the camera to cool down if it malfunctions.
3. If the message still appears, have your vehicle checked as soon as possible.

These steps can be found on pages 124 and 597 of the provided context information.

%pip install ChatTTS

In [12]:
import ChatTTS
import torchaudio
from IPython.display import Audio

In [13]:
chat = ChatTTS.Chat()
chat.load(compile=False) # Set to True for better performance

use default LlamaModel for importing TELlamaModel error: No module named 'transformer_engine'


True

In [14]:
print(str(response))

The daytime running lamps can be switched on and off using the following steps:

1. Adjust the front camera's brightness by cleaning the windshield if the visibility is reduced.
2. Wait a short period for the camera to cool down if it malfunctions.
3. If the message still appears, have your vehicle checked as soon as possible.

These steps can be found on pages 124 and 597 of the provided context information.


In [15]:
wavs = chat.infer(str(response))

#torchaudio.save("output1.wav", torch.from_numpy(wavs[0]), 24000)

found invalid characters: {'2', '4', ':', '3', '1', "'", '\n', '7', '9', '5'}
text:   0%|                                                                                                                                               | 0/384(max) [00:00, ?it/s]We detected that you are passing `past_key_values` as a tuple and this is deprecated and will be removed in v4.43. Please use an appropriate `Cache` class (https://huggingface.co/docs/transformers/v4.41.3/en/internal/generation_utils#transformers.Cache)
/home/amd/anaconda3/envs/rocm6/lib/python3.12/site-packages/transformers/models/llama/modeling_llama.py:648: UserWarning: 1Torch was not compiled with memory efficient attention. (Triggered internally at ../aten/src/ATen/native/transformers/hip/sdp_utils.cpp:505.)
  attn_output = torch.nn.functional.scaled_dot_product_attention(
text:  35%|████████████████████████████████████████████████▌                                                                                        | 136/38

In [16]:
Audio(wavs[0], rate=24_000, autoplay=True)

In [17]:
# Query Test 0
response = query_engine.query("如何切换DM系统工作模式并请给出手册的相关页码？")
response.print_response_stream()

To switch to a different drive mode, refer to page 304 in the manual for detailed instructions.

In [18]:
wavs = chat.infer(str(response))
Audio(wavs[0], rate=24_000, autoplay=True)

found invalid characters: {'3', '4', '0'}
text:   9%|███████████▊                                                                                                                              | 33/384(max) [00:00, 99.36it/s]
code:  13%|█████████████████▏                                                                                                                     | 260/2048(max) [00:02, 108.30it/s]


## To-Do: Optimization the RAG

Refer to Basic Strategies: https://docs.llamaindex.ai/en/v0.10.19/optimizing/basic_strategies/basic_strategies.html